In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import string

import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform

import pickle
import os

In [ ]:
# Load
cd = os.getcwd()
path = os.path.dirname(cd)

with open(path+'//Data_DG.pkl', 'rb') as file:
    # Patterns in input layer
    Data = pickle.load(file)   
    # The number of data
    Datalen = pickle.load(file)    
    # Patterns of granule cells
    Data_DG = pickle.load(file)
    
    # Lengths of each patterns of granule cells
    Data_DG_len = pickle.load(file)
    Data_idx = pickle.load(file) # indices of patterns having proper lengths
    DataX_idx = pickle.load(file) # indices of patterns having proper lengths
    
    # input/output overlap values and indices
    OverI = pickle.load(file)
    OverO = pickle.load(file)
    Pair_idx = pickle.load(file)
    
    # input/output overlap values and indices for inputs having proper lengths
    OverI_v2 = pickle.load(file)
    OverO_v2 = pickle.load(file)
    Pair_idx_v2 = pickle.load(file)
    
with open(path+'//Data_final.pkl', 'rb') as file:
    # Filtered by proper length
    Filt_idata = pickle.load(file) # patterns in input layer
    Filt_gdata = pickle.load(file) # patterns of granule cells
    Filt_glen = pickle.load(file) # lengths of each patterns of granule cells
    F_DataLen = pickle.load(file) # the number of data
    
    # Index of CA3 patterns
    Af_py = pickle.load(file) # Pyramidal neurons
    Af_in = pickle.load(file) # Inhibitory neurons
    
    # 
    In_py_list = pickle.load(file)
    In_in_list = pickle.load(file)

In [ ]:
def choice_neurons(layer, N):
    n_index = []
    num = random.randrange(0,layer)
    #N_R = np.random.randint(1, N)
    for n in range(N):
        while num in n_index:
            num = random.randrange(0,layer)
        n_index.append(num)
    n_index.sort()
    return n_index

In [ ]:
with open('Figure_spiking_retrieval_F.pkl', 'rb') as file:
    network = pickle.load(file)
    FT = pickle.load(file)
    Voltage = pickle.load(file)

In [ ]:
FT_rh = []
for N in range(9):
    FT_rh_temp = FT[N][0]
    for n in range(9):
        FT_rh_temp = np.concatenate((FT_rh_temp, FT[N][n+1]), axis=1)
    FT_rh.append(FT_rh_temp)
    
for N in range(9):
    FT_rh[N] = FT_rh[N]

In [ ]:
final_list = [249, 423, 80, 422, 228, 219, 517, 111, 346, 509]

In [ ]:
for i, comp in enumerate(final_list):
    print(np.where(Filt_idata[:,comp]))

In [ ]:
CA3_idx_set = set()
for i, comp in enumerate(final_list):
    CA3_idx_set |= set(Af_py[comp])
print(len(CA3_idx_set))

Phase_N = 1
CA3_FT = FT_rh[6][:,240*(Phase_N):240*(Phase_N+1)]
CA3_FT_Flist = []
CA3_FT_idx = []

for i, comp in enumerate(CA3_idx_set):
    temp = np.where(CA3_FT[comp,:])[0]
    for j, comp2 in enumerate(temp):
        CA3_FT_Flist.append([i,comp2])
        CA3_FT_idx.append([i,comp])
        
for i, comp in enumerate(CA3_FT_Flist):
    CA3_FT_Flist[i] = [comp[0],comp[1]-120]

In [ ]:
for f in range(10):

    Phase_N = f
    CA3_FT = FT_rh[6][:,240*(Phase_N):240*(Phase_N+1)]
    CA3_FT_Flist = []
    CA3_FT_idx = []

    for i, comp in enumerate(CA3_idx_set):
        temp = np.where(CA3_FT[comp,:])[0]
        for j, comp2 in enumerate(temp):
            CA3_FT_Flist.append([i,comp2])
            CA3_FT_idx.append([i,comp])

    for i, comp in enumerate(CA3_FT_Flist):
        CA3_FT_Flist[i] = [comp[0],comp[1]-120]

    window = 30 # ms
    FR_matrix = np.zeros((len(final_list),120-window))

    for n in range(2400):
        for t in range(240):
            for i, comp in enumerate(final_list):
                if CA3_FT[n,t] !=0 and n in Af_py[comp]:
                    for T in range(120-window):   
                        if T <= t-120 < T+window:
                            FR_matrix[i,T] +=1

    for i, comp in enumerate(final_list):
        FR_matrix[i,:] = (FR_matrix[i,:]/(0.001*window))/len(Af_py[comp])

    color_map = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'orange', 'purple', 'brown']
    labels = list(string.ascii_uppercase)[:10]
    
    x = np.array(range(30,120))
    fig = plt.subplot(111)
    for n in range(10):
        plt.plot(x, FR_matrix[n,:], color=color_map[n])
    plt.legend(labels, loc='upper left', frameon=False, fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel('Time (ms)', fontsize=24)
    plt.ylabel('Firing rate (Hz)', fontsize=24)
    if f == 2:
        plt.title('Winner: G', fontsize=24, y=1.05)
    elif f == 4:
        plt.title('Winner: E', fontsize=24, y=1.05)
    elif f == 3:
        plt.title('Fail', fontsize=24, y=1.05)
    else:
        plt.title('Competition between CA3 engrams', fontsize=24, y=1.05)
    plt.xlim([30,120])
    plt.ylim([0,130])
    fig.spines['top'].set_visible(False)
    fig.spines['right'].set_visible(False)
    plt.savefig('Firing rate for each engram in CA3-%d.pdf' %f, bbox_inches='tight')
    plt.show()